In [1]:
import persistence_functions as pf

In [2]:
direction = [0,0,1]

In [3]:
n = 14
points = [[[n,n,n], [n,n,n]]for n in range( n +1)]

In [4]:
print(points)

[[[0, 0, 0], [0, 0, 0]], [[1, 1, 1], [1, 1, 1]], [[2, 2, 2], [2, 2, 2]], [[3, 3, 3], [3, 3, 3]], [[4, 4, 4], [4, 4, 4]], [[5, 5, 5], [5, 5, 5]], [[6, 6, 6], [6, 6, 6]], [[7, 7, 7], [7, 7, 7]], [[8, 8, 8], [8, 8, 8]], [[9, 9, 9], [9, 9, 9]], [[10, 10, 10], [10, 10, 10]], [[11, 11, 11], [11, 11, 11]], [[12, 12, 12], [12, 12, 12]], [[13, 13, 13], [13, 13, 13]], [[14, 14, 14], [14, 14, 14]]]


In [5]:
pre_edges = [
    # Format: [[index_i, index_j]]
    [1, 2],
    [2, 4],
    [11, 12],
    [1, 3],
    [4, 7],
    [13, 14],
    [3, 8],
    [6,7],
    [8,9],
    [5,6],
    [9,10],
    [1,11]
]

In [6]:
# reindexed_edges = pf.reindex_edges(pre_edges)

In [7]:
pre_formatted_edges = []
for edge in pre_edges:
    x,y = edge
    pre_formatted_edges.append([[x, y], [1,1,1]])

In [8]:
pre_vertices = pf.append_height_vertices(direction, points)

In [9]:
print(pre_vertices)

[[[0, 0, 0], 0, [0, 0, 0]], [[1, 1, 1], 1, [1, 1, 1]], [[2, 2, 2], 2, [2, 2, 2]], [[3, 3, 3], 3, [3, 3, 3]], [[4, 4, 4], 4, [4, 4, 4]], [[5, 5, 5], 5, [5, 5, 5]], [[6, 6, 6], 6, [6, 6, 6]], [[7, 7, 7], 7, [7, 7, 7]], [[8, 8, 8], 8, [8, 8, 8]], [[9, 9, 9], 9, [9, 9, 9]], [[10, 10, 10], 10, [10, 10, 10]], [[11, 11, 11], 11, [11, 11, 11]], [[12, 12, 12], 12, [12, 12, 12]], [[13, 13, 13], 13, [13, 13, 13]], [[14, 14, 14], 14, [14, 14, 14]]]


In [10]:
vertices = pf.format_vertices(pre_vertices)

In [11]:
edges = pf.format_edges(vertices, pre_formatted_edges)

edges---------------------------------
[[1, 2], [1, 1, 1]]
1
2
edges---------------------------------
[[2, 4], [1, 1, 1]]
2
4
edges---------------------------------
[[11, 12], [1, 1, 1]]
11
12
edges---------------------------------
[[1, 3], [1, 1, 1]]
1
3
edges---------------------------------
[[4, 7], [1, 1, 1]]
4
7
edges---------------------------------
[[13, 14], [1, 1, 1]]
13
14
edges---------------------------------
[[3, 8], [1, 1, 1]]
3
8
edges---------------------------------
[[6, 7], [1, 1, 1]]
6
7
edges---------------------------------
[[8, 9], [1, 1, 1]]
8
9
edges---------------------------------
[[5, 6], [1, 1, 1]]
5
6
edges---------------------------------
[[9, 10], [1, 1, 1]]
9
10
edges---------------------------------
[[1, 11], [1, 1, 1]]
1
11


In [12]:
graph = pf.process_graph(vertices, edges, direction)

dictionary of indices
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


In [13]:
filtration = pf.group_events_by_height(graph[0], graph[1])

# Filtration Algorithm Start

In [14]:
uf = pf.UnionFind(graph[0])

Initialized UnionFind:
parent: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
rank  : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]



In [15]:
# Horizontal Step
def horizontal_step(edges: list):
    for edge in edges:
        x ,y = edge['vertices']
        uf.union(x, y)

In [16]:
def vertical_step(edges: list, components: dict, mergers: dict):
    for edge in edges:
        x ,y = edge['vertices']
        roots = uf.union(x, y)
        rootX = roots[x]['root']
        rootY = roots[y]['root']
        rankX = roots[x]['rank']
        rankY = roots[y]['rank']
        if rootX != rootY:
            if rootX in components and rootY in components:
                print(f'Component with root {rootY} got merged.')
                print(rankX, rankY)
                if rankY > rankX:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]
                elif rankX > rankY:
                    mergers[rootX]=max(edge['height'])
                    del components[rootX]
                else:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]

In [17]:
def compute_components(vertices, old_components):
    components = old_components
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root not in components:
            components[root] = []
        if node not in components[root]:
            components[root].append(node)
    return components

In [18]:
def compute_new_births(vertices):
    new_components = []
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root == node:
            new_components.append(vertex)
    for component in new_components:
        print(f"New Component with root {component}.")
    return new_components

In [19]:
total_components = {}
mergers = {}
total_vertices = []
births = []
for height, stage in filtration.items():
    print('Height: ', height)
    horizontal_edges = stage['horizontal_edges']
    vertical_edges = stage['vertical_edges']
    vertices = stage['points']

    # We join the horizontal edges
    horizontal_step(horizontal_edges)
    
    # We join the vertical edges
    vertical_step(vertical_edges, total_components, mergers)
    total_vertices.extend(vertices)

    # We compute the new components

    print("Merged Components------------------------")
    current_components = compute_components(total_vertices, total_components)
    for root, comp in current_components.items():
        print(f"Component with root {root}: {comp}")
    print("Merged Components------------------------")

    total_components = current_components

    # We compute the horizontal connected components at a given stage, the unmerged are new components
    print("New Births----------------------------------------------")
    new_births = compute_new_births(vertices)
    births.extend(new_births)
    print("New Births----------------------------------------------")

Height:  0
Merged Components------------------------
Component with root 0: [0]
Merged Components------------------------
New Births----------------------------------------------
New Component with root {'coordinates': [0, 0, 0], 'original_index': 0, 'new_index': 0, 'height': 0, 'sign': 0}.
New Births----------------------------------------------
Height:  1
Merged Components------------------------
Component with root 0: [0]
Component with root 1: [1]
Merged Components------------------------
New Births----------------------------------------------
New Component with root {'coordinates': [1, 1, 1], 'original_index': 1, 'new_index': 1, 'height': 1, 'sign': 1}.
New Births----------------------------------------------
Height:  2
Union(1, 2):
 - root of 1 is 1
 - root of 2 is 2
Merged Components------------------------
Component with root 0: [0]
Component with root 1: [1, 2]
Merged Components------------------------
New Births----------------------------------------------
New Births-------

In [20]:
for birth in births:
    print(birth)

{'coordinates': [0, 0, 0], 'original_index': 0, 'new_index': 0, 'height': 0, 'sign': 0}
{'coordinates': [1, 1, 1], 'original_index': 1, 'new_index': 1, 'height': 1, 'sign': 1}
{'coordinates': [5, 5, 5], 'original_index': 5, 'new_index': 5, 'height': 5, 'sign': 1}
{'coordinates': [13, 13, 13], 'original_index': 13, 'new_index': 13, 'height': 13, 'sign': 1}


In [21]:
for key, merger in mergers.items():
    print(key, merger)

5 7


In [22]:
print(total_components)

{0: [0], 1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 13: [13, 14]}
